In [4]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils

import h5py

X_train = np.load("20170827_mawilab_flow_000features.pkl")
Y_train = np.load("20170827_mawilab_flow_000labels.pkl")

scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
scaler = Normalizer().fit(Y_train)
Y_train = scaler.transform(Y_train)

# Test Set 1 : 20170827_mawilab_flow_001
X_test1 = np.load("20170827_mawilab_flow_001features.pkl")
Y_test1 = np.load("20170827_mawilab_flow_001labels.pkl")
scaler = Normalizer().fit(X_test1)
X_test1 = scaler.transform(X_test1)
scaler = Normalizer().fit(Y_test1)
Y_test1 = scaler.transform(Y_test1)

# Test Set 2 : 20170827_mawilab_flow_002
X_test2 = np.load("20170827_mawilab_flow_002features.pkl")
Y_test2 = np.load("20170827_mawilab_flow_002labels.pkl")
scaler = Normalizer().fit(X_test2)
X_test2 = scaler.transform(X_test2)
scaler = Normalizer().fit(Y_test2)
Y_test2 = scaler.transform(Y_test2)

# Test Set 3 : 20170827_mawilab_flow_003
X_test3 = np.load("20170827_mawilab_flow_003features.pkl")
Y_test3 = np.load("20170827_mawilab_flow_003labels.pkl")
scaler = Normalizer().fit(X_test3)
X_test3 = scaler.transform(X_test3)
scaler = Normalizer().fit(Y_test3)
Y_test3 = scaler.transform(Y_test3)

# Test Set 4: 20170827_mawilab_flow_004
X_test4 = np.load("20170827_mawilab_flow_004features.pkl")
Y_test4 = np.load("20170827_mawilab_flow_004labels.pkl")
scaler = Normalizer().fit(X_test4)
X_test4 = scaler.transform(X_test4)
scaler = Normalizer().fit(Y_test4)
Y_test4 = scaler.transform(Y_test4)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test1 = np.reshape(X_test1, (X_test1.shape[0],X_test1.shape[1],1))
X_test2 = np.reshape(X_test2, (X_test2.shape[0],X_test2.shape[1],1))
X_test3 = np.reshape(X_test3, (X_test3.shape[0],X_test3.shape[1],1))
X_test4 = np.reshape(X_test4, (X_test4.shape[0],X_test4.shape[1],1))


In [5]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(16, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 16, 64)            256       
_________________________________________________________________
activation_4 (Activation)    (None, 16, 64)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [6]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test3, Y_test3) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_03.png')

Train on 38361 samples, validate on 42398 samples
Epoch 1/25
38361/38361 [==============================] - 3s 77us/step - loss: 0.5288 - acc: 0.7000 - val_loss: 0.5481 - val_acc: 0.6432
Epoch 2/25
38361/38361 [==============================] - 2s 62us/step - loss: 0.4734 - acc: 0.7174 - val_loss: 0.5394 - val_acc: 0.6507
Epoch 3/25
38361/38361 [==============================] - 2s 62us/step - loss: 0.4650 - acc: 0.7265 - val_loss: 0.5253 - val_acc: 0.6382
Epoch 4/25
38361/38361 [==============================] - 3s 69us/step - loss: 0.4588 - acc: 0.7288 - val_loss: 0.5348 - val_acc: 0.6410
Epoch 5/25
38361/38361 [==============================] - 2s 62us/step - loss: 0.4543 - acc: 0.7322 - val_loss: 0.5244 - val_acc: 0.6398
Epoch 6/25
38361/38361 [==============================] - 2s 60us/step - loss: 0.4487 - acc: 0.7364 - val_loss: 0.5241 - val_acc: 0.6715
Epoch 7/25
38361/38361 [==============================] - 2s 62us/step - loss: 0.4455 - acc: 0.7392 - val_loss: 0.5247 - val_acc